In [1]:
# Get Dataset
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import levene, ttest_ind, wilcoxon, mannwhitneyu
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, roc_auc_score, accuracy_score
from scipy.stats import shapiro
import xgboost as xgb

In [2]:
data = pd.read_csv('./DataExtract.csv')

In [3]:
data.head()

,pdc,num_ip_post,total_los_post,num_op_post,num_er_post,num_ndc_post,num_gpi6_post,adjust_total_30d_post,generic_rate_post,post_ip_flag,...,brand_cost,ratio_G_total_cost,numofgen_post,numofbrand_post,generic_cost_post,brand_cost_post,ratio_G_total_cost_post,pdc_80_flag,drug_class,patient_key
0,0.894444,0,0,8,0,17,3,6.966667,1.000000,0,...,239.699550,0.710850,17,0,4332.043215,0.000000,1.000000,1,*ANTICOAGULANTS*,121
1,0.333333,0,0,4,0,15,5,14.466667,0.101382,0,...,2984.927229,0.010155,2,13,196.359216,3001.501507,0.061403,0,*ANTIDIABETICS*,168
2,0.077778,0,0,32,0,11,7,3.833333,0.739130,0,...,0.000000,1.000000,10,1,203.603270,1154.818505,0.149882,0,*ANTICOAGULANTS*,267
3,0.333333,0,0,57,0,16,6,24.000000,0.958333,0,...,160.804799,0.291008,15,1,140.952640,265.057382,0.347165,0,*ANTICOAGULANTS*,468
4,0.866667,0,0,5,0,16,4,18.000000,0.888889,0,...,0.000000,1.000000,14,2,671.755173,735.661568,0.477297,1,*ANTIDIABETICS*,499


In [4]:
data.patient_key.nunique()

3550

In [5]:
data.shape #no duplicate records

(3550, 95)

In [6]:
data_pdc_pop = data.groupby(['drug_class', 'pdc_80_flag']).patient_key.nunique()

In [7]:
data_pdc_pop

drug_class        pdc_80_flag
*ANTICOAGULANTS*  0              1437
                  1               391
*ANTIDIABETICS*   0              1051
                  1               671
Name: patient_key, dtype: int64

In [8]:
data_pdc_region = data.groupby(['drug_class', 'pdc_80_flag', 'regionN']).patient_key.nunique().reset_index()

In [9]:
data.groupby(['drug_class', 'pdc_80_flag', 'sexN']).patient_key.nunique()

drug_class        pdc_80_flag  sexN
*ANTICOAGULANTS*  0            1       661
                               2       776
                  1            1       228
                               2       163
*ANTIDIABETICS*   0            1       486
                               2       565
                  1            1       376
                               2       295
Name: patient_key, dtype: int64

In [10]:
from scipy.stats import chi2_contingency

# Filter data for ANTIDIABETICS drug class
AD = data[data['drug_class'].str.contains('ANTIDIABETICS')]


In [ ]:
AD_v2 = AD.drop(columns = ['pdc_80_flag',  'drug_class', 'patient_key', 'pdc_cat', 'pdc'])
y = AD['pdc_80_flag']

### model 2: feature selection using Logistic Lasso Regression

In [170]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(AD_v2, y, test_size=0.2, random_state=42)

In [171]:
cat = ['age_grpn', 'regionn', 'sexn']
X_train_con2 = X_train2.drop(columns = cat).fillna(0)
X_train_cat2 = X_train2[cat]
X_test_con2 = X_test2.drop(columns = cat).fillna(0)
X_test_cat2 = X_test2[cat]

In [172]:
scaler = StandardScaler()
X_train_normalized2 = scaler.fit_transform(X_train_con2)
X_test_normalized2 = scaler.transform(X_test_con2)

In [173]:
X_train_normalized_df2 = pd.DataFrame(X_train_normalized2, columns = X_train_con2.columns)
X_test_normalized_df2 = pd.DataFrame(X_test_normalized2, columns = X_train_con2.columns)
X_train_normalized_df2.reset_index(drop=True, inplace=True)
X_train_cat2.reset_index(drop=True, inplace=True)
X_test_normalized_df2.reset_index(drop=True, inplace=True)
X_test_cat2.reset_index(drop=True, inplace=True)

In [174]:
X_train_normalized_df2.tail(3)

,num_ip_post,total_los_post,num_op_post,num_er_post,num_ndc_post,num_gpi6_post,adjust_total_30d_post,generic_rate_post,post_ip_flag,post_er_flag,...,numofgen,numofbrand,generic_cost,brand_cost,ratio_g_total_cost,numofgen_post,numofbrand_post,generic_cost_post,brand_cost_post,ratio_g_total_cost_post
1374,-0.269487,-0.178651,-0.374638,-0.329215,-0.833134,-0.730952,-0.330888,0.990640,-0.290262,-0.395185,...,-0.018001,-0.682009,0.277723,-0.367234,1.110007,-0.568545,-0.861408,-0.446864,-0.487602,1.364182
1375,-0.269487,-0.178651,2.081387,1.244845,2.738248,4.732521,5.281420,-0.147297,-0.290262,2.530460,...,2.363316,1.517351,1.353713,2.372928,-1.070221,2.584154,1.507994,2.708295,2.921986,-0.796610
1376,-0.269487,-0.178651,-0.486276,-0.329215,-0.021456,-0.482612,0.232716,0.990640,-0.290262,-0.395185,...,-0.018001,-0.462073,0.014375,-0.359757,0.939221,0.448455,-0.861408,0.494360,-0.487602,1.364182


In [175]:
X_train_normalized_df2.shape, X_train_cat2.shape, y_train2.shape

((1377, 87), (1377, 3), (1377,))

In [176]:
X_train_lasso = pd.concat([X_train_normalized_df2, X_train_cat2], axis=1)
X_test_lasso = pd.concat([X_test_normalized_df2, X_test_cat2], axis=1)

In [177]:
X_train_lasso.tail(3)

,num_ip_post,total_los_post,num_op_post,num_er_post,num_ndc_post,num_gpi6_post,adjust_total_30d_post,generic_rate_post,post_ip_flag,post_er_flag,...,brand_cost,ratio_g_total_cost,numofgen_post,numofbrand_post,generic_cost_post,brand_cost_post,ratio_g_total_cost_post,age_grpn,regionn,sexn
1374,-0.269487,-0.178651,-0.374638,-0.329215,-0.833134,-0.730952,-0.330888,0.990640,-0.290262,-0.395185,...,-0.367234,1.110007,-0.568545,-0.861408,-0.446864,-0.487602,1.364182,3,1,1
1375,-0.269487,-0.178651,2.081387,1.244845,2.738248,4.732521,5.281420,-0.147297,-0.290262,2.530460,...,2.372928,-1.070221,2.584154,1.507994,2.708295,2.921986,-0.796610,3,4,2
1376,-0.269487,-0.178651,-0.486276,-0.329215,-0.021456,-0.482612,0.232716,0.990640,-0.290262,-0.395185,...,-0.359757,0.939221,0.448455,-0.861408,0.494360,-0.487602,1.364182,3,2,2


In [178]:
y_train2.head()

3358    0
2018    1
2911    0
1418    0
1891    1
Name: pdc_80_flag, dtype: int64

In [179]:
lasso_log_cv = LogisticRegressionCV(
    Cs=10,                   # Range of C values (inverse of regularization strength)
    cv=5,              
    penalty='l1',             # L1 regularization (Lasso)
    solver='saga',            # saga solver supports L1 regularization
    scoring='roc_auc',             # You can use other metrics like 'accuracy', 'roc_auc', etc.
    max_iter=10000,           # Higher iteration count for convergence
    class_weight='balanced'   #algorithm gives more weight to the minority class during training.

)


In [180]:
lasso_log_cv.fit(X_train_lasso, y_train2)

LogisticRegressionCV(Cs=10, class_weight='balanced', cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=10000,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
           refit=True, scoring='roc_auc', solver='saga', tol=0.0001,
           verbose=0)

In [181]:
best_c = lasso_log_cv.C_[0]
print(best_c)

0.046415888336127774


In [182]:
coefficients = lasso_log_cv.coef_[0] 
# Find the indices of non-zero coefficients
non_zero_indices = np.where(coefficients != 0)[0]
feature_names = AD_v2.columns
non_zero_feature_names = feature_names[non_zero_indices]

In [183]:
print(non_zero_feature_names) 

Index(['num_gpi6_post', 'adjust_total_30d_post', 'generic_rate_post',
       'post_ip_flag', 'post_er_flag', 'post_rx_cost', 'post_total_cost',
       'regionn', 'idx_prodtypen', 'chronic_pain_fibro', 'copd', 'hepatitis',
       'hiv_aids', 'hypertension', 'smoking', 'pre_ip_cost', 'num_op',
       'adjust_total_30d', 'log_pre_op_cost', 'numofbrand_post'],
      dtype='object')


In [184]:
lasso_log_cv.coef_

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.51570728,  1.2480942 ,  0.1006324 , -0.01217826, -0.11387213,
         0.        ,  0.        , -0.01230711,  0.        , -0.04247261,
         0.        ,  0.        ,  0.        ,  0.        ,  0.11794515,
        -0.10158919,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.05862351,  0.        ,  0.05308696,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.02139697, -0.06044495,
        -0.00194873,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.00252876,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.00890456,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.15201884,  0. 

In [185]:
selected_cols = [ 'num_gpi6_post', 'adjust_total_30d_post', 'generic_rate_post',
       'post_ip_flag', 'post_er_flag', 'post_rx_cost', 'post_total_cost',
       'regionn', 'idx_prodtypen', 'chronic_pain_fibro', 'copd', 'hepatitis',
       'hiv_aids', 'hypertension', 'smoking', 'pre_ip_cost', 'num_op',
       'adjust_total_30d', 'log_pre_op_cost', 'numofbrand_post']

In [186]:
AD_col2 = AD[selected_cols]

## Downsampling

In [205]:
AD_col2.shape

(1722, 20)

In [206]:
AD_v2.shape

(1722, 90)

In [207]:
from sklearn.utils import resample
majority = AD[AD['pdc_80_flag'] == 0]
minority = AD[AD['pdc_80_flag'] == 1]

majority_downsampled = resample(
    majority,
    replace = False,
    n_samples = len(minority),
    random_state = 42
)

In [208]:
majority_downsampled.head()

,pdc,num_ip_post,total_los_post,num_op_post,num_er_post,num_ndc_post,num_gpi6_post,adjust_total_30d_post,generic_rate_post,post_ip_flag,...,brand_cost,ratio_g_total_cost,numofgen_post,numofbrand_post,generic_cost_post,brand_cost_post,ratio_g_total_cost_post,pdc_80_flag,drug_class,patient_key
959,0.166667,0,0,5,0,4,3,3.233333,1.000000,0,...,94.538455,0.540263,4,0,61.682808,0.000000,1.000000,0,*ANTIDIABETICS*,123227
2153,0.166667,0,0,9,1,7,5,5.133333,0.415584,0,...,360.205970,0.023758,4,3,25.094436,279.348202,0.082427,0,*ANTIDIABETICS*,291778
1339,0.166667,0,0,4,0,16,8,12.633333,1.000000,0,...,33.056269,0.764696,16,0,227.704051,0.000000,1.000000,0,*ANTIDIABETICS*,170537
1055,0.166667,0,0,8,0,8,7,7.600000,1.000000,0,...,0.000000,1.000000,8,0,1109.540708,0.000000,1.000000,0,*ANTIDIABETICS*,135636
88,0.500000,0,0,7,0,32,9,30.600000,1.000000,0,...,0.000000,1.000000,32,0,737.840542,0.000000,1.000000,0,*ANTIDIABETICS*,13636


In [209]:
majority_downsampled.shape

(671, 95)

In [210]:
downsampled_df = pd.concat([majority_downsampled, minority], axis = 0)

In [211]:
downsampled_v2 = downsampled_df.drop(columns = ['pdc_80_flag',  'drug_class', 'patient_key', 'pdc_cat', 'pdc'])

In [212]:
downsampled_v2.shape

(1342, 90)

In [213]:
y4 = downsampled_df['pdc_80_flag']

In [214]:
sample_weights = np.array([2.57 if label == 0 else 1 for label in y4])

In [215]:
X_train4, X_test4, y_train4, y_test4, weights_train, weights_test = train_test_split(downsampled_v2, y4, sample_weights, test_size=0.2, random_state=42)

In [216]:
cat = ['age_grpn', 'regionn', 'sexn']
X_train_con4 = X_train4.drop(columns = cat).fillna(0)
X_train_cat4 = X_train4[cat]
X_test_con4 = X_test4.drop(columns = cat).fillna(0)
X_test_cat4 = X_test4[cat]

In [217]:
scaler = StandardScaler()
X_train_normalized4 = scaler.fit_transform(X_train_con4)
X_test_normalized4 = scaler.transform(X_test_con4)

In [218]:
X_train_normalized_df4 = pd.DataFrame(X_train_normalized4, columns = X_train_con4.columns)
X_test_normalized_df4 = pd.DataFrame(X_test_normalized4, columns = X_train_con4.columns)
X_train_normalized_df4.reset_index(drop=True, inplace=True)
X_train_cat4.reset_index(drop=True, inplace=True)
X_test_normalized_df4.reset_index(drop=True, inplace=True)
X_test_cat4.reset_index(drop=True, inplace=True)
X_train_lasso4 = pd.concat([X_train_normalized_df4, X_train_cat4], axis=1)
X_test_lasso4 = pd.concat([X_test_normalized_df4, X_test_cat4], axis=1)

In [219]:
lasso_cv = LogisticRegressionCV(
    Cs=10,                   # Range of C values (inverse of regularization strength)
    cv=5,              
    penalty='l1',             # L1 regularization (Lasso)
    solver='liblinear',            # liblinear solver supports L1 regularization
    scoring='roc_auc',             # You can use other metrics like 'accuracy', 'roc_auc', etc.
    max_iter=1000,           # Higher iteration count for convergence
    fit_intercept = True,
    class_weight= None      #set this to None since weights are handled by sample weights
)


In [220]:
lasso_cv.fit(X_train_lasso4, y_train4, sample_weight = weights_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=1000,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
           refit=True, scoring='roc_auc', solver='liblinear', tol=0.0001,
           verbose=0)

In [221]:
best_c = lasso_cv.C_[0]
print(best_c)

0.046415888336127774


In [222]:
coefficients = lasso_cv.coef_[0] 
# Find the indices of non-zero coefficients
non_zero_indices = np.where(coefficients != 0)[0]
feature_names = downsampled_v2.columns
non_zero_feature_names = feature_names[non_zero_indices]

In [223]:
print(non_zero_feature_names) 

Index(['num_gpi6_post', 'adjust_total_30d_post', 'generic_rate_post',
       'post_ip_flag', 'post_op_cost', 'post_medical_cost', 'regionn',
       'log_idx_copay', 'alcohol_drug', 'asthma', 'cardiac_valvular',
       'cerebrovascular', 'chronic_kidney', 'chronic_pain_fibro', 'depression',
       'hepatitis', 'hiv_aids', 'hypertension', 'solid_tumor', 'pre_ip_cost',
       'num_op', 'adjust_total_30d', 'pre_er_flag', 'log_pre_ip_cost',
       'log_pre_op_cost', 'pre_total_cat', 'generic_cost', 'generic_cost_post',
       'brand_cost_post', 'ratio_g_total_cost_post'],
      dtype='object')


In [224]:
selected_cols = ['num_gpi6_post', 'adjust_total_30d_post', 'generic_rate_post',
       'post_ip_flag', 'post_op_cost', 'post_medical_cost', 'regionn',
       'log_idx_copay', 'alcohol_drug', 'asthma', 'cardiac_valvular',
       'cerebrovascular', 'chronic_kidney', 'chronic_pain_fibro', 'depression',
       'hepatitis', 'hiv_aids', 'hypertension', 'solid_tumor', 'pre_ip_cost',
       'num_op', 'adjust_total_30d', 'pre_er_flag', 'log_pre_ip_cost',
       'log_pre_op_cost', 'pre_total_cat', 'generic_cost', 'generic_cost_post',
       'brand_cost_post', 'ratio_g_total_cost_post']

In [225]:
selected_X_train5 = X_train4.loc[:, selected_cols]

In [226]:
selected_X_test5 = X_test4.loc[:, selected_cols]

## model 4 with example weights

In [227]:
# Create DMatrix from training data
dtrain5 = xgb.DMatrix(selected_X_train5, label=y_train4, weight = weights_train)

cv_results = xgb.cv(
    params = params3,
    dtrain = dtrain5,
    nfold = 5,
    num_boost_round = 10,
    verbose_eval = 2
)

[0]	train-logloss:0.64994+0.00089	test-logloss:0.66406+0.00557
[2]	train-logloss:0.58165+0.00102	test-logloss:0.62122+0.01185
[4]	train-logloss:0.52990+0.00146	test-logloss:0.59355+0.01655
[6]	train-logloss:0.48875+0.00202	test-logloss:0.57300+0.02159
[8]	train-logloss:0.45554+0.00249	test-logloss:0.55736+0.02385
[9]	train-logloss:0.44076+0.00250	test-logloss:0.55084+0.02637


In [228]:
best_logloss5 = cv_results['test-logloss-mean'].min()
best_num_boost_round5 = cv_results['test-logloss-mean'].idxmin() + 1

In [229]:
print(best_num_boost_round5)

10


In [232]:
model4 = xgb.train(
    params = params3, 
    dtrain = dtrain5, 
    num_boost_round = best_num_boost_round5)
dtest5 = xgb.DMatrix(selected_X_test5)
y_pred_prob5 = model4.predict(dtest5)

## Model 4 Evaluation

In [238]:
y_pred5 = np.where(y_pred_prob5 > 0.5, 1, 0)

In [239]:
print(classification_report(y_test4, y_pred5))
AUC = roc_auc_score(y_test4, y_pred5)
print(f'AUC score: {AUC}')

             precision    recall  f1-score   support

          0       0.69      0.74      0.71       137
          1       0.71      0.66      0.68       132

avg / total       0.70      0.70      0.70       269

AUC score: 0.698158593231586


In [240]:
f1_scorer = make_scorer(f1_score)
# classifier1 is the machine learning model

# Perform 5-fold cross-validation with F1 score as the scoring metric
f1_scores = cross_val_score(classifier1, selected_X_train5, y_train4, cv=5, scoring=f1_scorer)

# Calculate the mean F1 score across all folds
mean_f1_score = f1_scores.mean()
print("Mean F1 Score:", mean_f1_score)

Mean F1 Score: 0.7107265370412745
